In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import livenet
importlib.reload(livenet)
core = livenet.core
import simple_log
simple_log.level = simple_log.LogLevel.INFO
from simple_log import LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



In [ ]:
downscale = (14, 14)
train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_mnist_train(), downscale=downscale, to_odd=True, to_gray=True)
n_hidden = 2

# downscale = (8, 8)
# train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_cifar10_train(), downscale=downscale, to_odd=False, to_gray=True)
# n_hidden = 12

n_out = np.max(train_y.numpy()) + 1



In [ ]:
context = core.livenet.Context()
network = livenet.nets.create_perceptron(train_x.shape[1], n_hidden, n_out, context)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=1)

In [ ]:
network.context.regularization_l1 = 0.01
optimizer.learning_rate = 0.001
trainer.step(5000)


In [ ]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff): .3f}"


In [ ]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000. + ord(x[0][1]) / 1e6):
    print(name, f"{p.item():.1f}")



In [ ]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["N1->D0"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_picker1)
plt.grid()
plt.plot(values)

#accum = core.stat_utils.AccumStat()
#accum.add_value(trainer.network.parameters())
#accum.plot()


In [ ]:
network._parameters


In [ ]:
core.livenet.export_onnx(network, "/home/spometun/model.onnx")
